# Libs

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
from scipy import spatial
import gensim
from gensim.models import Word2Vec 
# import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


In [130]:
#pip install stop-words

# Read files

In [45]:
def read_files(file_loc):
    df = pd.ExcelFile(file_loc)
    df = df.parse('Sheet1')
    return df

desc = read_files(r"C:\Users\Aishwarya\Desktop\ISB task\OneDrive_1_01-10-2019\company descriptions.xlsx")
keyw = read_files(r"C:\Users\Aishwarya\Desktop\ISB task\OneDrive_1_01-10-2019\Industry Segments - Top 10 Keywords.xlsx")

## Data treatment

In [46]:
desc['company_description'] = desc['company_description'].str.strip()
desc['company_short_description'] = desc['company_short_description'].str.strip()


desc['company_description'].replace( { r'[^A-Za-z ]+' : '' }, inplace= True, regex = True)
desc['company_short_description'].replace( { r'[^A-Za-z ]+' : '' }, inplace= True, regex = True)
desc = desc.replace(np.nan, '', regex = True) # Replace nans with ''
desc['company_description'] = np.where(desc['company_description'] == '', desc['company_short_description'], desc['company_description'])
desc['company_description'] = np.where(desc['company_description'].str.len() < 25, desc['company_short_description'] + desc['company_description'], desc['company_description'])

desc['company_description'] = desc['company_description'].str.replace('http\S+|www.\S+', '', case=False)

keyw['Top 10 keywords'].replace( { r'[^A-Za-z ]+' : '' }, inplace= True, regex = True)
keyw = keyw.replace(np.nan, '', regex = True)

# Load pre-trained vectors from Gensim 

In [5]:
from gensim.models import KeyedVectors

# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format("C:\word2vec\GoogleNews-vectors-negative300.bin.gz", binary=True, limit = 250000) # limiting to 250000 words to avoid memory issues and slow runtimes. Library actually has 3 mil words

In [121]:
def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in word2vec_model.wv.vocab]
    if len(words) >= 1:
        return np.mean(word2vec_model[words], axis=0)
    else:
        return []

In [131]:
vecs = np.zeros((desc.shape[0],300))
err = []

for i in range(0,desc.shape[0]):
    comp_desc = desc.iloc[i,2]

    data = []
    # tokenize the sentence into words 
    for j in word_tokenize(comp_desc):
        if j not in stop_words:
            data.append(j.lower())
    try:
        vecs[i] = get_mean_vector(model, data) 
        err.append(1)
    except:
        err.append(0)

In [99]:
vecs_all = []

for i in range(0,desc.shape[0]):
    vecs_all.append(vecs[i].tolist())

comps = pd.DataFrame(vecs_all)
comps['list_vec'] = comps.values.tolist()
comps = comps['list_vec'].to_frame()
comps['company'] = desc["company_name"]

In [104]:
vecs = np.zeros((keyw.shape[0],300))
err = []

In [132]:
for i in range(0,keyw.shape[0]):
    comp_desc = keyw.iloc[i,1]

    data = []
    # tokenize the sentence into words 
    for j in word_tokenize(comp_desc):
        if j not in stop_words:
            data.append(j.lower())
    try:
        vecs[i] = get_mean_vector(model, data)
        err.append(1)
    except:
        err.append(0)

In [124]:
vecs_all = []

for i in range(0,keyw.shape[0]):
    vecs_all.append(vecs[i].tolist())

segs = pd.DataFrame(vecs_all)
segs['list_vec_segs'] = segs.values.tolist()
segs = segs['list_vec_segs'].to_frame()
segs['Industry segment'] = keyw["Industry segment"]

# Matching

In [126]:
comps['key'] = 1
segs['key'] = 1

cross = pd.merge(comps, segs, on = 'key')


In [ ]:
cross['cosine_sim'] = 0

for i in range(0, cross.shape[0]):
    vec1 = cross.iloc[i,0]
    vec2 = cross.iloc[i,3]
    result = 1 - spatial.distance.cosine(vec1, vec2)
    cross.iloc[i,5] = result

In [ ]:
df_fin = pd.merge(cross, cross.groupby(['company'])["cosine_sim"].max(), on = ["company", "cosine_sim"])

In [ ]:
df_fin[['company', 'Industry segment', 'cosine_sim']].to_csv("classify_pretrained_vectors.csv")